In [1]:
import pandas as pd 

In [2]:
import os 

In [3]:
path = "archive"

# Dataset 

In [4]:
rating = pd.read_csv(path + "/" + "rating.csv")
tag = pd.read_csv(path + "/" + "tag.csv")
movie = pd.read_csv(path + "/" + "movie.csv")


In [5]:

rating

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [6]:
movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [7]:

tag

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,2009-04-24 18:19:40
1,65,208,dark hero,2013-05-10 01:41:18
2,65,353,dark hero,2013-05-10 01:41:19
3,65,521,noir thriller,2013-05-10 01:39:43
4,65,592,dark hero,2013-05-10 01:41:18
...,...,...,...,...
465559,138446,55999,dragged,2013-01-23 23:29:32
465560,138446,55999,Jason Bateman,2013-01-23 23:29:38
465561,138446,55999,quirky,2013-01-23 23:29:38
465562,138446,55999,sad,2013-01-23 23:29:32


# Features 

**movies**

In [9]:
movie["_genres"] =  movie["genres"].apply(lambda x: x.split("|")[0])

In [10]:
_movie = movie[["movieId", "_genres"]]
_movie

,movieId,_genres
0,1,Adventure
1,2,Adventure
2,3,Comedy
3,4,Comedy
4,5,Comedy
...,...,...
27273,131254,Comedy
27274,131256,Comedy
27275,131258,Adventure
27276,131260,(no genres listed)


**tags**

In [11]:
_tag = tag.groupby(['userId','movieId']).first().reset_index()[["userId",'movieId','tag']]
_tag

,userId,movieId,tag
0,18,4141,Mark Waters
1,65,208,dark hero
2,65,353,dark hero
3,65,521,noir thriller
4,65,592,dark hero
...,...,...,...
174839,138446,3489,Peter Pan
174840,138446,7045,family friendly
174841,138446,7164,Peter Pan
174842,138446,55999,dragged


**genome_tags**

In [12]:

genome_scores = pd.read_csv(path + "/" + "genome_scores.csv")
genome_tags = pd.read_csv(path + "/" + "genome_tags.csv")


In [13]:
genome_tags

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [14]:
genome_scores

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675
...,...,...,...
11709763,131170,1124,0.58775
11709764,131170,1125,0.01075
11709765,131170,1126,0.01575
11709766,131170,1127,0.11450


In [15]:
_genome = pd.merge(genome_scores, genome_tags, on = 'tagId')


In [16]:
_genome = _genome.groupby('movieId', group_keys=False).apply(lambda x: x.loc[x.relevance.idxmax()]).reset_index(drop=True)

In [17]:
_genome = pd.merge(_genome, genome_tags, on = ['tagId','tag'], how = 'left').rename(columns = 
                                                                                    {"tag":"genome_tag",
                                                                                    "relevance": "genome_relevance"}
                                                                                   
                                                                                   ).drop(columns = {
    "tagId"
})
_genome

,movieId,genome_relevance,genome_tag
0,1,0.99925,toys
1,2,0.98100,adventure
2,3,0.97450,good sequel
3,4,0.97675,women
4,5,0.96575,good sequel
...,...,...,...
10376,130578,0.88325,assassination
10377,130840,0.96500,romance
10378,131013,0.98425,comedy
10379,131168,0.97300,betrayal


**concat features**

In [18]:
_genome

,movieId,genome_relevance,genome_tag
0,1,0.99925,toys
1,2,0.98100,adventure
2,3,0.97450,good sequel
3,4,0.97675,women
4,5,0.96575,good sequel
...,...,...,...
10376,130578,0.88325,assassination
10377,130840,0.96500,romance
10378,131013,0.98425,comedy
10379,131168,0.97300,betrayal


In [19]:
_tag

,userId,movieId,tag
0,18,4141,Mark Waters
1,65,208,dark hero
2,65,353,dark hero
3,65,521,noir thriller
4,65,592,dark hero
...,...,...,...
174839,138446,3489,Peter Pan
174840,138446,7045,family friendly
174841,138446,7164,Peter Pan
174842,138446,55999,dragged


In [20]:
_movie

,movieId,_genres
0,1,Adventure
1,2,Adventure
2,3,Comedy
3,4,Comedy
4,5,Comedy
...,...,...
27273,131254,Comedy
27274,131256,Comedy
27275,131258,Adventure
27276,131260,(no genres listed)


In [21]:
rating = pd.read_csv(path + "/" + "rating.csv")
rating

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [22]:

rating = pd.merge(rating, _movie, on ='movieId', how = 'left')
rating

,userId,movieId,rating,timestamp,_genres
0,1,2,3.5,2005-04-02 23:53:47,Adventure
1,1,29,3.5,2005-04-02 23:31:16,Adventure
2,1,32,3.5,2005-04-02 23:33:39,Mystery
3,1,47,3.5,2005-04-02 23:32:07,Mystery
4,1,50,3.5,2005-04-02 23:29:40,Crime
...,...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00,Adventure
20000259,138493,69526,4.5,2009-12-03 18:31:48,Action
20000260,138493,69644,3.0,2009-12-07 18:10:57,Action
20000261,138493,70286,5.0,2009-11-13 15:42:24,Mystery


In [23]:
rating = pd.merge(rating, _tag, on =['movieId','userId'], how = 'left')
rating

,userId,movieId,rating,timestamp,_genres,tag
0,1,2,3.5,2005-04-02 23:53:47,Adventure,NaN
1,1,29,3.5,2005-04-02 23:31:16,Adventure,NaN
2,1,32,3.5,2005-04-02 23:33:39,Mystery,NaN
3,1,47,3.5,2005-04-02 23:32:07,Mystery,NaN
4,1,50,3.5,2005-04-02 23:29:40,Crime,NaN
...,...,...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00,Adventure,NaN
20000259,138493,69526,4.5,2009-12-03 18:31:48,Action,NaN
20000260,138493,69644,3.0,2009-12-07 18:10:57,Action,NaN
20000261,138493,70286,5.0,2009-11-13 15:42:24,Mystery,NaN


In [24]:
rating= pd.merge(rating, _genome, on =["movieId"], how = "left")
rating

,userId,movieId,rating,timestamp,_genres,tag,genome_relevance,genome_tag
0,1,2,3.5,2005-04-02 23:53:47,Adventure,NaN,0.98100,adventure
1,1,29,3.5,2005-04-02 23:31:16,Adventure,NaN,0.99800,dark fantasy
2,1,32,3.5,2005-04-02 23:33:39,Mystery,NaN,0.99825,future
3,1,47,3.5,2005-04-02 23:32:07,Mystery,NaN,0.99675,serial killer
4,1,50,3.5,2005-04-02 23:29:40,Crime,NaN,0.99875,oscar (best supporting actor)
...,...,...,...,...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00,Adventure,NaN,0.99475,pixar animation
20000259,138493,69526,4.5,2009-12-03 18:31:48,Action,NaN,0.99850,giant robots
20000260,138493,69644,3.0,2009-12-07 18:10:57,Action,NaN,0.98500,animation
20000261,138493,70286,5.0,2009-11-13 15:42:24,Mystery,NaN,0.99900,alien


In [25]:
rating["tag"] = rating["tag"].fillna("None")
rating["_genres"] = rating["_genres"].fillna("None")
rating["genome_tag"] = rating["genome_tag"].fillna("None")
rating["genome_relevance"] = rating["genome_relevance"].fillna(0)
rating


,userId,movieId,rating,timestamp,_genres,tag,genome_relevance,genome_tag
0,1,2,3.5,2005-04-02 23:53:47,Adventure,None,0.98100,adventure
1,1,29,3.5,2005-04-02 23:31:16,Adventure,None,0.99800,dark fantasy
2,1,32,3.5,2005-04-02 23:33:39,Mystery,None,0.99825,future
3,1,47,3.5,2005-04-02 23:32:07,Mystery,None,0.99675,serial killer
4,1,50,3.5,2005-04-02 23:29:40,Crime,None,0.99875,oscar (best supporting actor)
...,...,...,...,...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00,Adventure,None,0.99475,pixar animation
20000259,138493,69526,4.5,2009-12-03 18:31:48,Action,None,0.99850,giant robots
20000260,138493,69644,3.0,2009-12-07 18:10:57,Action,None,0.98500,animation
20000261,138493,70286,5.0,2009-11-13 15:42:24,Mystery,None,0.99900,alien


In [27]:
rating._genres.nunique()

20

In [28]:
rating.genome_tag.nunique()

813

In [30]:
rating.tag.nunique()

17941

In [26]:
rating.to_csv("manipulated_rating.csv", index=False)